##  감성체인 생성

In [1]:
import os
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.chains import LLMChain  # LLMChain을 사용하여 Chain을 구성합니다
import logging

In [2]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0.3)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate.from_template(
    """你是一名研究中国的社会科学家，你的任务是分析中国新闻对“美日韩”三边关系的报道，并从中国的视角分析这些报道的情感**。

    **评分标准**：
    - 强烈负面 (-5): 极端消极
    - 负面 (-3 ~ -4): 较为消极
    - 轻微负面 (-1 ~ -2): 略微消极
    - 中立 (0): 纯粹的事实陈述，不带有任何情感倾向。
    - 轻微正面 (+1 ~ +2): 略微积极
    - 正面 (+3 ~ +4): 较为积极
    - 强烈正面 (+5): 极端积极

    **请严格分析以下新闻文章的所有句子，并为每个句子提供一个情感分数。 句子应根据句号（。）进行分隔，不得遗漏任何句子**。
    **新闻文章内容:**
    {article_content}

    **请返回每个句子的情感分数，每行一个分数，不要包含任何额外的文本、注释或解释。**
    """
)

In [4]:
# 체인 구성
sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)

C:\Users\82104\AppData\Local\Temp\ipykernel_2092\1462471065.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)


## 데이터 로드

In [6]:
import json

# JSON 파일 로드
file_path = "../data/0310/split_by_year/2010.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 {len(data)}개의 기사 로드 완료")


총 5개의 기사 로드 완료


## 감성분석 수행 함수

In [7]:
def analyze_sentiment(article):
    """기사 전체 본문을 입력하고 문장별 감성 점수를 분석하는 함수"""
    try:
        # 기사 본문만 전달
        input_data = {"sentences": article["sentences"]}

        # `sentiment_chain.run()`을 사용하여 데이터 처리
        response = sentiment_chain.invoke(input_data)  # input_data 전달

        print(f"Response format: {response}")

        # 'text' 키에서 감성 분석 결과를 추출하여 'sentiment_responses'에 저장
        if isinstance(response, dict):
            if 'text' in response:
                article["sentiment_responses"] = response['text'].strip()  # 감성 분석 결과 저장
            else:
                article["sentiment_responses"] = "Error: 'text' key not found in response"
        else:
            article["sentiment_responses"] = "Error: Response is not a dictionary"

        return article
    except Exception as e:
        article["sentiment_responses"] = f"Error: {str(e)}"  # 오류 발생 시 오류 메시지 저장
        return article  # 오류 발생 시에도 응답 내용 저장

def process_article(article):
    """GPT 응답 후처리"""
    sentiment_response = article.get("sentiment_responses", "")

    # 문장이 없는 경우 바로 반환
    if not sentiment_response:
        return article

    sentiment_scores = []
    for line in sentiment_response.split("\n"):
        try:
            # 문장별 감성 점수를 처리
            sentiment_scores.append(float(line.strip()))  # 각 점수 추가
        except (ValueError, IndexError):
            continue
    article["sentiment_scores"] = sentiment_scores
    article["avg_sentiment_score"] = round(sum(sentiment_scores) / len(sentiment_scores), 2) if sentiment_scores else None
    return article

def rerun_analysis_for_null_avg(data, output_file):
    """avg_sentiment_score가 null인 데이터만 다시 분석하는 함수"""
    results = []

    for article in data:
        if article.get("avg_sentiment_score") is None:  # avg_sentiment_score가 null인 경우
            print(f"Re-analyzing article: {article['title']}")  # 다시 분석할 기사 출력
            # 감성 분석을 다시 실행하고 후처리
            article = analyze_sentiment(article)
            article = process_article(article)

        # 결과 저장
        results.append(article)

        # 중간 저장 (기사별로 즉시 JSON 파일 업데이트)
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=4)

    print(f" Re-analysis completed for articles with null avg_sentiment_score. Results saved to {output_file}.")


In [8]:
output_path = "../../data/0310/results/result_2006-2009.json"
run_sentiment_analysis(data, output_path)

2025-03-10 11:15:57,045 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-10 11:15:57,048 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '7月5日朝鲜连续发射7枚导弹引起国际社会震惊。近日美日在安理会积极推动制裁朝鲜的议案并加紧协调进一步的行动方案。朝鲜此举再一次将自身推向国际政治风云的风口浪尖。 朝鲜这次以一些国 朝鲜此次发射导弹可能给东北亚地区局势增加更加复杂和难以预料的因素。日本已明确表示要提前部署爱国者3地空拦截导弹。韩国担忧日本会以此为借口趁机加速军事化步伐。日本政府在去年已经投入12亿美元用于建立导弹防御系统并且作为日美联合计划的一部分。日本开始从美国采购相关武器。日本还计划2010年向太空发射八颗间谍卫星用于配合导弹防御系统。 朝鲜发射导弹后美日韩澳俄英法等国纷纷发表声明称朝鲜此举是挑衅行为破坏了亚太的稳定。尤其美日反应强烈但事态进一步升级为武装冲突的可能性不大。日美在安理会推动的制裁议案通过的几率微乎其微。有人预测朝鲜急切想通过试射导弹达成目的的想法可能适得其反美国将会进一步收紧对朝孤立政策日朝关系正常化将更为困难。日本已经考虑单方面对朝鲜实施包括经济制裁在内的反制措施。 朝鲜问题只有相关各方都表现出诚意和建设性危机才能化为解决问题的契机。', 'text': '-2\n-1\n-2\n-2\n-2\n-2\n-2\n-1\n-3\n-2\n-1\n-2\n0'}
Response format: {'article_content': '被外界视为破冰之旅的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常化进行了两天紧张的会谈。尽管双方还没有向外披露过多的会谈结果但外界普遍认为美朝此次双边会谈具有重要意义将有可能改善美朝之间的关系并对推动各方落实六方会谈的有关协议具有一定的积极作用。 此次美朝双边会谈的主要议题是商讨如何落实2月13日北京朝核问题六方会谈达成的协议以及两国关系正常化等问题。希尔6日在会谈结束后表示美国与朝鲜在纽约有关实现两国关系正常化的会谈很成功。美朝双方代表都对在60天内实现朝鲜关闭其主要核反应堆以换取能源援助的目标感到乐观。 金桂冠在会谈前表示美国与朝鲜关系取得进展对于美国总统布什与国务卿赖斯来说将是一大政绩希望双方会谈不受朝核六方会谈形式的拘泥就是要与美方自由交换意见而不是针锋相对地谈判。金桂冠5日在与希尔正式会谈前还会见了前国务卿基辛格和奥尔布赖特等美国前政要以及朝

2025-03-10 11:16:01,504 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '自上个月朝鲜宣布发射人造卫星以来美国日本和韩国高举制裁大棒对朝鲜的贸易活动进行了进一步的封锁并减少了对朝援助这使朝鲜的经济发展再次面临困境。为使经济恢复活力朝鲜全国目前正在掀起一场名为150天战斗的自救行动。 农业和能源是两大重点 据韩国媒体报道朝鲜劳动党在最高人民会议代议员选举结束的第二天即3月10日向劳动党员下发了告全体党员书提出了2012年建设社会主义强盛大国的宏伟蓝图并指明了具体的生产目标每年生产776万瓦电3300万吨金属1300万吨煤700万吨粮食实现经济现代化技术集约化。为进一步贯彻党中央的这一精神朝鲜劳动党于上个月末下达了开展150天战斗的指示。 报道说所谓150天战斗是从5月5日朝鲜发射火箭一个月开始到10月10日朝鲜劳动党建党纪念日结束其优先目标是建设住房努力使农业铁路正常化救活地方产业工厂和煤矿。在实现这些目标的基础上再提高铁路金属工业生活必需品等生产。此前被外界视为金正日热门接班人选的三子金正云将担任这场经济运动的总指挥。 韩国媒体认为救活农业和煤矿是朝鲜目前亟待解决的问题。由于对外援助的减少朝鲜今年的粮食短缺问题将更加严重。朝鲜最高领导人金正日近日特地下达指示要求全民总动员提高谷类产量在粮食短缺问题上实现决定性的逆转。能源不足也是朝鲜面临的一大难题它将直接影响到年底朝鲜居民的供暖。因此150天战斗将把恢复煤矿生产作为一项重点。 分析人士认为受援助中断和经济制裁的影响朝鲜此次开展的150天战斗比较注重实际集中解决的都是一些短期能够见效的问题譬如修复工厂企业的生产线完成规定的生产量等体现了朝鲜解决当前经济危机的决心。 靠自救摆脱经济困境 据悉朝鲜过去每当经济遇到困难时为了寻找经济突破口都会开展限时性的自救运动。最具代表性的是1978年开展的100天战斗和1988年开展的200天战斗。在200天战斗期间朝鲜建设了发电量达100千瓦左右的中小水电站开启了沙里院肥料第一阶段工程金策制钢厂扩充工程等庞大的项目。 自朝鲜4月5日宣布发射卫星后朝鲜半岛和平进程再次出现波折。朝鲜宣布退出六方会谈并重启核设施美日韩三国对朝关系重现紧张态势相关的援助也相应中断。在此情况下朝鲜不得不靠自己的努力走出经济困境。 为实现2012年的强国目标朝鲜把经济建设作为今年的工作重点。新年伊始朝鲜中央媒体

2025-03-10 11:16:46,035 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '神秘这是朝鲜留给外界的印象。 正因为神秘一些国家的情报机构费尽心机通过投放卫星派遣侦察机甚至空降特工等手段洞察朝鲜的任何蛛丝马迹。这些国家当中尤以美国韩国和日本三国情报机构最为活跃。 近来朝鲜最高领导人金正日的健康问题再次牵动美韩日三国情报机关的敏感神经。有媒体报道称这些国家的情报部门已全速运转密切注视着朝鲜境内的任何风吹草动三国派遣的特工更是云集朝鲜周边重重谍云笼罩朝鲜。 美日韩编织谍报大网 朝鲜地缘位置敏感环境又相对封闭以至朝鲜许多本应正常公开的信息都成为这些国家竞相搜集的情报。而朝鲜出于安全考虑对此更是守口如瓶。除了金正日的健康状况还有诸如粮食产量钢铁产量能源需求等重大经济民生信息以及高层人事变动核研发动向国内外政策调整等战略性信息都是三国情报人员猎取的对象。由于朝鲜对外界不透明美日韩三国间谍聚集朝鲜周边也在所难免。据了解三国情报机构一直有着紧密合作以美国中央情报局韩国国家情报院以及日本内阁情报调查室为主以美国国家安全局韩国军方情报部门以及日本防卫厅情报本部为辅三国已经在朝鲜境内外编织了一张密集的谍报大网。 法国回声报就曾披露自1993年3月朝鲜核风云骤起以来美国就与日本韩国一道对朝鲜进行了全天候的立体监视。三国也因此花费不薄。此后一年多美国就耗资3亿多美元日本和韩国的支出也各超过了2亿美元。据称近年来韩国为搜集朝鲜军事情报所花费的资金已占其情报总开支的60。 空中侦察地面窃听 对美日韩来说朝鲜就犹如密不透风的铁桶要派遣特工深入其境内不仅风险极大且成功率也较低。因此依靠高科技手段获取朝鲜情报成为三国的首选。综合运用卫星图片及传感设备分析朝鲜情报是三国的王牌。1994年以中情局为首的美国情报系统开始对朝鲜核发展情况展开系统的研究。为了摸清朝鲜核项目的底细美国家安全局翻出了近20年来间谍卫星在朝鲜上空拍摄的所有照片中情局也找出自己保密箱内沉睡多年的各种资料并根据这些照片资料及其他来源的情报分析朝鲜的核进展。2005年美国宣布朝鲜有可能进行核试验其依据就是通过对比前几年目标区的卫星图片发现在可能的核试验区域有新的设备进入以及管线铺设的迹象由此判断朝鲜在积极准备核试验。空中侦察也是三国对朝刺探情报的重要手段。据简氏防务周刊介绍担负侦察朝鲜任务的是驻冲绳美空军第390情报中队的3架RC135战略

2025-03-10 11:16:49,979 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '改进型大浦洞2号据说可攻击美本土甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关系恶化的警告后朝鲜半岛形势突然紧张起来。一直在密切注意朝鲜动向的美日韩三国政府近日搜集到朝鲜可能发射远程导弹的情报这可急坏了三国的政府官员。由于朝鲜此前有试射失败的先例因此他们担心如果朝鲜导弹再次偏离目标将带来十分可怕的后果。 为应对朝鲜可能再度试射导弹美日韩正严密监控预警。图为韩国民众在战争博物馆参观导弹模型。 朝方可能散发假情报 韩国政府消息人士近日表示朝鲜运至咸镜北道花台郡舞水端里导弹基地的大浦洞2号导弹的试射时间可能提前。消息人士称2006年朝鲜在将导弹运至基地2个月后进行了试射但此次朝鲜可能加固了导弹弹体和发射台因此试射时间可能会提前。 韩国方面认为如果朝鲜加快脚步最快会在本月16日即朝鲜最高领导人金正日生日当天发射导弹因为韩国民间团体计划在金正日生日前后用气球向朝鲜散发朝鲜货币和反朝传单朝鲜一直对此深感愤怒。 此前韩国媒体曾预测说发射大浦洞2号导弹在第一二三阶段火箭组装发射台搭建液体燃料注入等方面需要相当长的时间。因此韩国情报当局推测朝鲜在一两个月之后才有可能发射导弹也就是说很可能选在4月25日朝鲜人民军建军日发射。 不过韩国的情报是否准确也值得怀疑因为朝鲜非常清楚它的军事设施暴露在美韩的间谍卫星和其他侦察设施之下所以经常通过各种手段进行伪装从而造成韩美误判。实际上韩国政府消息人士本月3日还宣称朝鲜正往平安北道铁山郡东仓里的发射实验场运送导弹。韩国方面认为造成情报前后矛盾的原因是朝鲜方面数次变更列车的行进路线。 美日严密监控防御 尽管朝鲜是否将发射导弹仍不能确定但是美日韩三国已经制定了充分的应对措施。美军太平洋司令基廷近日表示如果朝鲜试射导弹只要总统一声令下我们将以多种方式进行应对。驻韩美军司令沃尔特夏普4日声称韩美两国将在乌山设立联合空军司令部以应对可能爆发的战争。 据报道美国海军为了防御朝鲜的导弹攻击在太平洋地区集中部署了16艘具有弹道导弹防御功能的巡洋舰和驱逐舰。美国的宙斯盾级导弹驱逐舰也奉命驶往朝鲜附近海域。目前美军的U2高空侦察机RC135侦察机KH11侦察卫星以及韩军的RF4侦察机三八线附近的电子战装备也都全部开动全面监视朝鲜的一举一动。 在侦察朝鲜方面自然少不了日本。据日本媒体报道